In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

# for class imbalance
from imblearn.over_sampling import SMOTE

### Baseline Model

In [24]:
df = pd.read_csv('final_df.csv', index_col=0)
df.head(3)

,track_id,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,time_signature
0,10.0,-17.238174,94.442024,-48.614548,32.370766,-4.550584,2.220590,-2.271865,2.682117,-5.288854,...,2.239940,-6.416430,2.677109,-5.104513,-0.911098,-3.207996,3.043154,-1.036512,3.882601,4.0
1,141.0,-208.569260,116.633934,-1.723712,26.018599,9.688706,-4.179668,7.235491,-5.119752,-0.052033,...,-3.396803,-4.810870,3.130367,-2.126665,5.053833,-0.170090,3.087436,-8.665928,-1.098168,3.0
2,213.0,-264.020782,160.387070,0.499303,15.058126,12.012090,-8.957160,-10.785332,0.959413,-8.229311,...,-5.370893,-4.490613,-11.134265,-18.518158,-2.583431,-4.884125,1.371706,4.428988,-3.584380,4.0


In [25]:
df['time_signature'].value_counts()

4.0    1034
3.0     128
5.0      42
1.0      17
0.0       1
Name: time_signature, dtype: int64

In [26]:
# dropping all rows where the time signature is 1 or 0
df = df[(df['time_signature'] != 0) & (df['time_signature'] != 1)]

In [27]:
df['time_signature'].value_counts()

4.0    1034
3.0     128
5.0      42
Name: time_signature, dtype: int64

In [30]:
X = df.drop(columns=['track_id', 'time_signature'])
y = df['time_signature']

In [36]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
dummy_clf.predict(X)
dummy_clf.score(X, y)

0.8588039867109635

In [39]:
y_train.value_counts(normalize=True)

4.0    0.859813
3.0    0.106957
5.0    0.033229
Name: time_signature, dtype: float64

#### Train test split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=12)

In [35]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(963, 20)
(241, 20)
(963,)
(241,)


### Upsampling the smaller classes

In [48]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [49]:
print(X_res.shape)
print(y_res.shape)

(2484, 20)
(2484,)


In [55]:
pd.DataFrame(y_res)[0].value_counts(normalize=True)

3.0    0.333333
5.0    0.333333
4.0    0.333333
Name: 0, dtype: float64

### Knn Model

In [60]:
knn = KNeighborsClassifier()
knn.fit(X_res, y_res)
knn_preds = knn.predict(X_test)
accuracy_score(y_test, knn_preds)

0.5020746887966805